# Retrieval-Augmented Generation

## Einführung

In diesem Notebook wird mit LangGraph ein Chatbot für die *Magpie* implementiert. LangGraph ist ein Framework, das auf LangChain basiert und speziell für zustandsbasierte, mehrstufige KI-Agenten in Form von Graphen entwickelt wurde.

LangGraph ermöglicht die Erstellung von komplexen Entscheidungsstrukturen und Workflows für KI-Agenten. Es unterstützt Schleifen, Verzweigungen und wiederkehrende Prozesse. Ein zentrales Element ist die Zustandsverwaltung, wodurch Agenten über längere Interaktionen hinweg Kontext behalten und aktualisieren können.

Der Chatbot unterscheidet sich im Vorgehen von einer *normalen* RAG: Während bei einer RAG üblicherweise Text generiert wird (die Anfrage durch einen User), der gegen eine Vektordatenbank durchsucht werden kann, besteht der Ansatz für strukturierte Daten darin, dass das LLM SQL-Abfragen schreibt und ausführt.

Der zu generierende Agent soll folgendermaßen vorgehen: 

1. Passende Variable für die Anfrage finden
   - Gehe in die Tabelle `variable`.
   - Suche in der Spalte `beschr`.
   - Nutze das entsprechende Retriever-Tool.
   - Bestimme die ID der passenden Variable.

2. Passende Reichweite für die Anfrage finden
   - Gehe in die Tabelle `reichweite`.
   - Suche in der Spalte `beschr`.
   - Nutze das entsprechende Retriever-Tool.
   - Bestimme die ID der passenden Reichweite.

3. Passende Werteinheit für die Anfrage finden
   - Gehe in die Tabelle `wert_einheit`.
   - Suche in der Spalte `beschr`.
   - Nutze das entsprechende Retriever-Tool.
   - Bestimme die ID der passenden Werteinheit.

4. Daten filtern und Antwort generieren
   - Gehe in die Tabelle `daten`.
   - Filtere die Daten nach den gefundenen IDs.
   - Gib die Antwort aus.

## Durchführung

### Arbeitspfad definieren

In einem ersten Schritt definieren wir unseren Arbeitspfad:

In [ ]:
# Aktuelles Arbeitsverzeichnis ermitteln
import os
os.getcwd()
os.chdir("c:/Users/Hueck/OneDrive/Dokumente/GitHub/magpie_langchain")

### LLM laden

In einem weiteren Schritt laden wir ein Large Language Modell (LLM). Das LLM wandelt die Anfrage des Users in eine SQL-Abfrage um und antwortet basierend auf den Ergebnissen der Abfrage in natürlicher Sprache. Das sich `Mistral`, `qwen2.5:32b` und  `llama3.1:8b-instruct-q4_0` nicht so leistungsfähig wie `gpt-4o-mini` im Kontext der Generierung der SQL-Abfragen erwiesen hat, wird vorerst mit einer OpenAI-API gearbeitet. Sie wird mit `load_dotenv()` im folgenden aus dem Environment geladen um auf `gpt-4o-mini` zugreifen zu können. Der Code importiert weiterhin die `ChatOpenAI`-Klasse aus dem Modul `langchain_openai`, welches die Nutzung von OpenAI-Modellen vereinfacht. Anschließend wird ein Objekt `llm` vom Typ `ChatOpenAI` erstellt, wobei das Modell `"o3-mini"` angegeben wird.

In [ ]:
from dotenv import load_dotenv

# Lade Umgebungsvariablen aus der .env Datei
load_dotenv()

from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o")

# from langchain_ollama import ChatOllama
# llm = ChatOllama(model="qwen2.5:32b", temperature=0)

### Tools genieren

Wir erstellen nun ein Set von standardisierten `tools`. Das `SQLDatabaseToolkit` umfasst in diesem Kontext Werkzeuge, die uns dabei unterstützen, SQL-Abfragen zu erstellen und auszuführen sowie die Syntax von SQL-Abfragen zu überprüfen. 

Es wird dafür eine Instanz von `SQLDatabase` erstellt, um Interaktionen mit der Datenbank zu ermöglichen. Nach der Einrichtung der Datenbank wird eine Instanz von `SQLDatabaseToolkit` erstellt, die zwei Argumente benötigt: die zuvor erstellte `db`-Instanz und die oben definierte `llm` (Sprachmodell)-Instanz (siehe oben). Das Toolkit nutzt das Sprachmodell für Aufgaben wie die Validierung von Abfragen.

Die Methode `get_tools` der `SQLDatabaseToolkit`-Instanz wird dann aufgerufen, um eine Liste der verfügbaren Werkzeuge im Toolkit abzurufen. Diese Werkzeuge umfassen Funktionen wie das Auflisten von Datenbanktabellen, das Abrufen von Schema-Informationen, das Ausführen von SQL-Abfragen und das Überprüfen der Korrektheit von SQL-Abfragen. Die letzte Zeile des Codes gibt die Liste der Werkzeuge aus, sodass der Benutzer die verfügbaren Werkzeuge und deren Beschreibungen einsehen kann.

In [ ]:
from langchain_community.agent_toolkits import SQLDatabaseToolkit
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("duckdb:///data/magpie.db")
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

tools = toolkit.get_tools()

for tool in tools:
    print(f"Tool Name: {tool.name}")
    print(f"Description: {tool.description}")
    print("-" * 40)

### Retriever Tools

Wir erzeugen für die Schritte 1-3 drei verschiedene Retriever-Tools. Ziel ist, dass diese zur Anfrage passende Reichweiten und Variablen in der Magpie finden und entsprechende Werteinheiten für die Interpretation einholen. 

Die Retriever gehen dabei wie folgt vor:

1. Zuerst werden die `beschr` als Liste abgefragt und bereinigt.
2. Anschließend werden aus diesen Texten Embeddings mit OpenAI erstellt und in einem In-Memory-Vektor-Store gespeichert.
3. Dieser Vektor-Store dient als Grundlage für die Retriever, die passend zur Anfrage Begriffe anhand einer Vektor-Suche finden kann.
4. Abschließend wird aus dem Retriever ein Tool generiert, das später vom Chatbot genutzt werden kann, um passende Begriffe zu identifizieren.

Wir genieren in dieser Form folgende Tools:

1. `rt_beschr_variable`
2. `rt_beschr_reichweite`
3. `rt_beschr_wert_einheit`



In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore
from langchain.agents.agent_toolkits import create_retriever_tool
import ast
import re

##################################################################
# Generiere `rt_beschr_variable`
##################################################################

def query_as_list(db, query):
    res = db.run(query)
    res = [el for sub in ast.literal_eval(res) for el in sub if el]
    res = [re.sub(r"\b\d+\b", "", string).strip() for string in res]
    return list(set(res))


beschr_variable = query_as_list(db, "SELECT beschr FROM variable")

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

vector_store = InMemoryVectorStore(embeddings)

_ = vector_store.add_texts(beschr_variable)

retriever = vector_store.as_retriever(search_kwargs={"k": 5})

description = (
    "Use to look up values to filter on. Input is an approximate spelling "
    "of the proper noun, output is valid proper nouns. Use the noun most "
    "similar to the search."
)

rt_beschr_variable = create_retriever_tool(
    retriever,
    name="rt_beschr_variable",
    description=description,
)

##################################################################
# Generiere `rt_reichweite_variable`
##################################################################

reichweite_variable = query_as_list(db, "SELECT beschr FROM Reichweite")

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

vector_store = InMemoryVectorStore(embeddings)

_ = vector_store.add_texts(reichweite_variable)

retriever = vector_store.as_retriever(search_kwargs={"k": 5})

description = (
    "Use to look up values to filter on. Input is an approximate spelling "
    "of the proper noun, output is valid proper nouns. Use the noun most "
    "similar to the search."
)

rt_reichweite_variable = create_retriever_tool(
    retriever,
    name="rt_reichweite_variable",
    description=description,
)

##################################################################
# Generiere `rt_beschr_wert_einheit`
##################################################################

werteinheit_variable = query_as_list(db, "SELECT beschr FROM wert_einheit")

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

vector_store = InMemoryVectorStore(embeddings)

_ = vector_store.add_texts(werteinheit_variable)

retriever = vector_store.as_retriever(search_kwargs={"k": 5})

description = (
    "Use to look up values to filter on. Input is an approximate spelling "
    "of the proper noun, output is valid proper nouns. Use the noun most "
    "similar to the search."
)

rt_werteinheit_variable = create_retriever_tool(
    retriever,
    name="rt_beschr_wert_einheit",
    description=description,
)

NameError: name 'Tool' is not defined

Wir prüfen unsere Tools:

In [ ]:
queries = [
  ("englischer Studiengang", rt_beschr_variable),
  ("FU Berlin", rt_reichweite_variable),
  ("EUR", rt_werteinheit_variable)
]

for query, tool in queries:
    output = tool.invoke(query)
    print(f"\nErgebnisse von {tool.name} für Anfrage '{query}':")
    for i, item in enumerate(output.split("\n"), 1):
        if item.strip():  # Leere Zeilen ignorieren
            print(f"{i}. {item.strip()}")



In [50]:
from langchain_core.tools import tool

@tool
def get_variable_id(description: str) -> str:
    """
    Sucht in der Tabelle 'variable' nach einer passenden ID basierend auf der Beschreibung.
    Nutzt das bestehende Retrieval-Tool, um Tippfehler oder ungenaue Eingaben zu korrigieren.
    """
    # Verwende das bestehende Retriever-Tool, um die beste Übereinstimmung zu finden
    matches = rt_beschr_variable.invoke(description)
    print(matches)
    if not matches:
        return "Error: Keine passende Variable gefunden."

    best_match = matches.split("\n")[0]  # Nimm das relevanteste Ergebnis, also das. 1.

    # SQL-Abfrage mit der gefundenen besten Übereinstimmung
    query = f"SELECT id FROM variable WHERE beschr = '{best_match}' LIMIT 1;"
    result = db.run_no_throw(query)

    return result if result else "Error: Keine passende Variable gefunden."


@tool
def get_reichweite_id(description: str) -> str:
    """
    Sucht in der Tabelle 'reichweite' nach einer passenden ID basierend auf der Beschreibung.
    Nutzt das bestehende Retrieval-Tool, um Tippfehler oder ungenaue Eingaben zu korrigieren.
    """
    # Verwende das bestehende Retriever-Tool, um die beste Übereinstimmung zu finden
    matches = rt_reichweite_variable.invoke(description)
    if not matches:
        return "Error: Keine passende Variable gefunden."

    best_match = matches.split("\n")[0]  # Nimm das relevanteste Ergebnis, also das. 1.

    # SQL-Abfrage mit der gefundenen besten Übereinstimmung
    query = f"SELECT id FROM Reichweite WHERE beschr = '{best_match}' LIMIT 1;"
    print(query)
    result = db.run_no_throw(query)

    return result if result else "Error: Keine passende Variable gefunden."

@tool
def get_wert_einheit_id(description: str) -> str:
    """Sucht in der Tabelle 'wert_einheit' nach einer passenden ID basierend auf der Beschreibung."""
    query = f"SELECT id FROM wert_einheit WHERE beschr LIKE '%{description}%' LIMIT 1;"
    result = db.run_no_throw(query)
    return result if result else "Error: Keine passende Werteinheit gefunden."

@tool
def get_data(variable_id: str, reichweite_id: str, wert_einheit_id: str) -> str:
    """Extrahiert Daten aus der Tabelle 'daten' basierend auf den IDs der Variable, Reichweite und Werteinheit."""
    query = f"""
        SELECT wert, zeit_start, zeit_ende 
        FROM daten 
        WHERE variable_id = '{variable_id}' 
        AND reichweite_id = '{reichweite_id}' 
        AND id = '{wert_einheit_id}'
        ORDER BY jahr;
    """
    result = db.run_no_throw(query)
    return result if result else "Error: Keine passenden Daten gefunden."

tools.extend([get_variable_id, get_reichweite_id, get_wert_einheit_id, get_data])


## Manueller Probelauf



In [47]:
question = "Nenne mir die Anzahl der Menschen, die 2019 ihr Studium ohne Abitur angefangen haben?"


#get_variable_id.invoke("Nenne mir die Anzahl der Menschen, die 2019 ihr Studium ohne Abitur angefangen haben?")
#variable id: 123

get_reichweite_id.invoke("Nenne mir die Anzahl der Menschen, die 2019 ihr Studium ohne Abitur angefangen haben?")
rt_reichweite_variable.invoke("Nenne mir die Anzahl der Menschen, die 2019 ihr Studium ohne Abitur angefangen haben?")

SELECT id FROM Reichweite WHERE beschr = 'Sonstige Hochschule' LIMIT 1;


'Sonstige Hochschule\n\nSonstiges Orientierungsstudium\n\nHochschulsektor\n\nOrientierungsstudium MINT\n\nUniversität Rostock (ohne Klinikum)'

### Prompt Template generieren 

Im folgenden wird das `langchain`-Paket verwendet, um ein Prompt-Template von `langchain-ai` zu laden und zu überprüfen. Zuerst wird die `hub`-Funktion aus dem `langchain`-Paket importiert, um auf Modelle und Vorlagen im LangChain Hub zuzugreifen. Mit `hub.pull("langchain-ai/sql-agent-system-prompt")` wird ein spezifisches Prompt-Template abgerufen, in diesem Fall das `sql-agent-system-prompt`. `sql-agent-system-prompt` ist ein vordefiniertes Prompt-Template. Es dient als Vorlage für die Interaktion mit einer SQL-Datenbank über ein Sprachmodell. Dieses Template ist speziell für die Verwendung mit SQL-Agenten konzipiert, die SQL-Abfragen generieren oder validieren können.

Der Code prüft weiterhin mit `assert len(prompt_template.messages) == 1`, ob das geladene Template genau eine Nachricht enthält. Falls dies nicht zutrifft, wird ein Abbruch ausgelöst. Abschließend wird mit `prompt_template.messages[0].pretty_print()` die Nachricht im Template in einem gut lesbaren Format ausgegeben, um sicherzustellen, dass der Inhalt korrekt geladen wurde.

In [ ]:
from langchain import hub

prompt_template = hub.pull("langchain-ai/sql-agent-system-prompt")

assert len(prompt_template.messages) == 1, "Die Anzahl der Nachrichten im Template ist nicht 1!"
# Bearbeite die bestehende Nachricht, indem du Text hinzufügst
prompt_template.messages[0].prompt.template += (
    "\nYou are Sparklehorse, a chatbot for the Stifterverband organization. "
    "Your primary task is to answer questions related to the Magpie database."
)

prompt_template.messages[0].pretty_print()

Wir befüllen nun die Objekte, die in `prompt_template` bisher nur als Platzhalter definiert sind. Diese sind:
1. `dialect`
2. `top_k`

 `dialect` meint den SQL-Dialekt der verwendeten Datenbank. `top_k` bestimmt die Anzahl der zurückgegebenen, *besten Ergebnisse*. In diesem Kontext bedeutet *beste Ergebnisse* die Auswahl der relevantesten oder nützlichsten Ergebnisse der Abfragen, etwa bzgl. der Relevanz im Zusammenhang mit einer Abfrage.


In [ ]:
system_message = prompt_template.format(
    dialect=db.dialect, 
    top_k=5
)

print(system_message)

### Agent generieren    

Es wird nun eine Instanz eines *React-Agenten* aus der `langchain_core` und `langgraph` Bibliothek erstellt und konfiguriert. Im einzelnen passiert das Folgende: Die Funktion `create_react_agent(llm, tools, state_modifier=system_message)` erstellt einen neuen Agenten, der auf das bereitgestellte Sprachmodell (`llm`) und eine Sammlung von Werkzeugen (`tools`) zugreifen kann, um Aufgaben zu erledigen. Das Sprachmodell (`llm`) ist die Instanz eines Modells `gpt-4o-mini`, das vom Agenten verwendet wird, während tools die Werkzeuge sind, mit denen der Agent auf Daten zugreifen oder Aktionen durchführen kann, zum Beispiel eine SQL-Datenbankabfrage oder APIs. `state_modifier=system_message` gibt eine Systemnachricht an, die die Rolle des Agenten und seine Aufgaben innerhalb des Systems beschreibt, einschließlich der Anweisungen, wie der Agent mit den Eingaben des Benutzers umgehen soll. Die erstellte Instanz des React-Agenten wird in der Variablen `agent_executor` gespeichert, die dann verwendet wird, um Interaktionen mit einem Benutzer zu führen und die entsprechenden Werkzeuge anzuwenden.

In [ ]:
from langchain_core.messages import HumanMessage
from langgraph.prebuilt import create_react_agent

# Systemnachricht mit extra Anweisungen
suffix = (
    "Before generating an SQL query, you must first: \n"
    "1. Use 'get_variable_id' to get the correct variable ID.\n"
    "2. Use 'get_reichweite_id' to get the correct reichweiten-ID. Entscheide selbst, ob das nötig ist\n"
    "3. Use 'get_wert_einheit_id' to get the correct value unit.\n"
    "4. retrieve the relevant data aus dem Tabelle daten.\n"
    "NEVER attempt to guess values—always use these tools."
)

system = f"{system_message}\n\n{suffix}"

# Neuen ReAct-Agent erstellen mit den vollständigen Tools
agent_executor = create_react_agent(llm, tools, state_modifier=system)



## Probelauf Chatbot

Schließlich testen wir den Bot: Zunächst wird dafür eine Frage (`question`) definiert: *"Wie hoch waren die Drittmittel der FU Berlin im Jahr 2006 insgesamt?"*. Diese Frage wird später an den Agenten übergeben.

Dann wird eine Schleife (`for step in agent.stream(...)`) gestartet, die den Agenten Schritt für Schritt durch den Verarbeitungsprozess führt.  

- Der Agent erhält die Eingabe als eine Nachricht im Format `{"role": "user", "content": question}`.  
- Das Argument `stream_mode="values"` sorgt dafür, dass die Antwort in einzelnen Schritten ausgegeben wird.  
- Innerhalb der Schleife wird der jeweils letzte Schritt (`step["messages"][-1]`) formatiert und ausgegeben, sodass die Antwort für den Nutzer lesbar bleibt.  

Dadurch wird der Verarbeitungsweg der Antwort des Agenten schrittweise angezeigt.

In [52]:
# Testanfrage an den Agenten
question = "Nenne mir die Anzahl der Menschen, die 2019 ihr Studium ohne Abitur angefangen haben?"

for step in agent_executor.stream(
    {"messages": [HumanMessage(content=question)]}, 
    stream_mode="values"
):
    step["messages"][-1].pretty_print()


================================ Human Message =================================

Nenne mir die Anzahl der Menschen, die 2019 ihr Studium ohne Abitur angefangen haben?
================================== Ai Message ==================================
Tool Calls:
  get_variable_id (call_VgeRUz88GV5CXDJWre4vHSqF)
 Call ID: call_VgeRUz88GV5CXDJWre4vHSqF
  Args:
    description: Personen, die 2019 Studium ohne Abitur angefangen haben
Anzahl der Studienanfänger ohne Abitur

Anteil der Studienanfänger ohne Abitur

Studienabsolventen ohne Abitur

Anteil der Studienabsolventen ohne Abitur

Anzahl der Studienanfänger duales Studium
================================= Tool Message =================================
Name: get_variable_id

[(123,)]
================================== Ai Message ==================================
Tool Calls:
  get_reichweite_id (call_yxtro9NqMkCEd5y4XiQ5maIw)
 Call ID: call_yxtro9NqMkCEd5y4XiQ5maIw
  Args:
    description: Deutschland
SELECT id FROM Reichweite WHERE besc

In [ ]:
# IDs dynamisch abrufen
variable_id = db.run_no_throw("SELECT variable_id FROM variable WHERE beschr = 'Anzahl der Studienanfänger ohne Abitur' LIMIT 1;")
reichweite_id = db.run_no_throw("SELECT id FROM reichweite WHERE beschr = '2019' LIMIT 1;")
werteinheit_id = db.run_no_throw("SELECT id FROM werteinheit WHERE beschr = 'Anzahl' LIMIT 1;")



# Prüfen, ob alle IDs gefunden wurden
if variable_id and reichweite_id and werteinheit_id:
    # Query mit LEFT JOIN und den IDs
    query = f"""
        SELECT d.wert, rw.beschr
        FROM daten d
        LEFT JOIN daten_reichweite dr ON d.id = dr.daten_id
        LEFT JOIN reichweite rw ON dr.reichweite_id = rw.id
        LEFT JOIN reichweite_typ rt ON rw.reichweite_typ_id = rt.id
        WHERE d.variable_id = '{variable_id[0][0]}'
        AND rw.id = '{reichweite_id[0][0]}'
        AND d.werteinheit_id = '{werteinheit_id[0][0]}';
    """
    result = db.run_no_throw(query)
    print(result if result else "Error: Keine passenden Daten gefunden.")
else:
    print("Error: Eine oder mehrere IDs konnten nicht gefunden werden.")


In [ ]:
variable_id